In [9]:
import pandas as pd
import os
import s3fs

# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

BUCKET = "cgadeau/Antoine Airdrop/"
FILE_KEY_S3 = "df_all_clusters_final.csv"
FILE_PATH_S3 = BUCKET + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    dataset = pd.read_csv(file_in, sep=",")

In [10]:
with fs.open(BUCKET + "df_Données_Reabos_odd_final.csv", mode='rb') as file_in:
    bigdata = pd.read_csv(file_in, sep=',')
print(bigdata['ID_ABONNE'].nunique())

780796


In [7]:
print(dataset['ID_ABONNE'].nunique())
dataset

78079


,ID_ABONNE,DATE_ACTE_REEL,DATE_DEMARRAGE_PROMO,TYPE_PROMON,STATUT_FIN_M_MOINS_1,DATE_PRISE_EFFET,DATE_FIN_ABO_PREC,DATE_FIN_REABO,REGION,FORMULE_PREC,...,MOYEN_PAIEMENT,REABO_AVANT_ECHEANCE,DELAI_REABO,NB_APPARITIONS,MOY_DELAI,PREMIERE_APPARITION,ANCIENNETE,NOMBRE_ABONNEMENTS,SCORE_FIDELITE,Cluster_8
0,51276418001,2022-06-19,2022-06-19,ODD 30 jours TC,1.0,2022-06-19,2022-05-31,2022-09-18,Sénégal - Divers,AFRIQUE - EVASION & CANAL+,...,Cash,0,19,9,17.111111,2021-02-16,986,"[5, 4]",0.346154,4
1,51276418001,2021-12-01,2021-12-01,ODD 21 jours TC,0.0,2021-12-01,2021-08-23,2022-02-28,Sénégal - Divers,AFRIQUE - EVASION,...,Cash,0,100,9,17.111111,2021-02-16,986,"[5, 4]",0.346154,4
2,51276418001,2021-05-15,2021-05-15,ODD 15 jours TC,1.0,2021-05-15,2021-04-30,2021-06-14,Dakar,AFRIQUE - EVASION & CANAL+,...,Cash,0,15,9,17.111111,2021-02-16,986,"[5, 4]",0.346154,4
3,51276418001,2022-01-16,2022-01-16,ODD 15 jours TC,1.0,2022-03-01,2022-02-28,2022-05-31,Sénégal - Divers,AFRIQUE - EVASION & CANAL+,...,Cash,1,-43,9,17.111111,2021-02-16,986,"[5, 4]",0.346154,4
4,51276418001,2021-07-24,2021-07-24,ODD 15 jours TC,0.0,2021-07-24,2021-06-14,2021-08-23,Dakar,AFRIQUE - EVASION & CANAL+,...,Cash,0,40,9,17.111111,2021-02-16,986,"[5, 4]",0.346154,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946857,51458332904,2021-02-09,2021-02-09,PAS_ODD,1.0,2021-02-09,2021-02-07,2022-02-08,Dakar,AFRIQUE - EVASION & CANAL+,...,Cash,0,2,3,3.666667,2021-02-09,993,"[1, 1, 1]",0.076923,4
946858,51458332904,2023-02-16,2023-02-16,PAS_ODD,1.0,2023-02-16,2023-02-14,2024-02-15,Dakar,AFRIQUE - EVASION & CANAL+,...,Cash,0,2,3,3.666667,2021-02-09,993,"[1, 1, 1]",0.076923,4
946859,51720983201,2021-09-11,2021-10-01,PAS_ODD,1.0,2021-10-01,2021-09-30,2022-09-30,Dakar,AFRIQUE - ACCESS,...,Prélèvement RIB,1,-19,3,-19.000000,2021-09-11,779,"[1, 1, 1]",0.076923,4
946860,51720983201,2022-09-11,2022-10-01,PAS_ODD,1.0,2022-10-01,2022-09-30,2023-09-30,Dakar,AFRIQUE - ACCESS,...,Prélèvement RIB,1,-19,3,-19.000000,2021-09-11,779,"[1, 1, 1]",0.076923,4


In [4]:
def date_bascule(df, cluster):
    data = df.copy()
    data = data.loc[data["Cluster_8"] == cluster]
    data['DATE_PRISE_EFFET'] = pd.to_datetime(data['DATE_PRISE_EFFET'])
    data['ANNEE_MOIS'] = data['DATE_PRISE_EFFET'].dt.strftime('%Y-%m')
    result = data.groupby('ANNEE_MOIS')['ID_ABONNE'].nunique().reset_index()
    result.rename(columns={'ID_ABONNE' : 'NOMBRE_ABONNEMENTS'}, inplace = True)

    def ecart_type_avant_après(dataframe, line):
        result_before = dataframe.iloc[line:]
        result_after = dataframe.iloc[:line]
        std_before = result_before['NOMBRE_ABONNEMENTS'].std()
        mean_before = result_before['NOMBRE_ABONNEMENTS'].mean()
        std_after = result_after['NOMBRE_ABONNEMENTS'].std()
        mean_after = result_after['NOMBRE_ABONNEMENTS'].mean()
        return (std_before, std_after, mean_before, mean_after)

    min_std = float('inf')
    date = None
    min_std_before = None
    min_std_after = None
    the_mean_before = None
    the_mean_after = None
    for line in range(1, result.shape[0]):
        std_before, std_after, mean_before, mean_after = ecart_type_avant_après(result, line)
        std_line = std_before + std_after
        if min_std > std_line:
            min_std = std_line
            min_std_before = std_before
            min_std_after = std_after
            the_mean_before = mean_before
            the_mean_after = mean_after
            date = line
    
    ratio = the_mean_before/the_mean_after
    if ratio > 19/20 and ratio < 20/19 :
        date = None
        min_std_before = None
        min_std_after = None
        the_mean_before = None
        the_mean_after = None

    if date != None :
        mois_bascule = result.loc[date, 'ANNEE_MOIS']
        print('Date = ', mois_bascule)
    else :
        print('Date = ', date)
    print('Std error before = ', min_std_before)
    print('Std error after = ', min_std_after)
    print('Mean before = ', the_mean_before)
    print('Mean after = ', the_mean_after)
    print('Ecart des moyennes = ')

    return(result, mois_bascule)

liste_bascule = []
for i in range (0, 8, 1):
    print(i)
    liste_bascule += [date_bascule(dataset, i)[1]]
print(liste_bascule)

0
Date =  2021-09
Std error before =  1314.2953566894257
Std error after =  38.18376618407357
Mean before =  2388.730769230769
Mean after =  29.0
Ecart des moyennes = 
1
Date =  2022-07
Std error before =  419.1377607660756
Std error after =  772.4214789714683
Mean before =  1311.0625
Mean after =  3624.6666666666665
Ecart des moyennes = 
2
Date =  2021-07
Std error before =  2442.5056857932336
Std error after =  1716.4049250298328
Mean before =  4346.892857142857
Mean after =  3240.3333333333335
Ecart des moyennes = 
3
Date =  2021-05
Std error before =  1470.3906103911427
Std error after =  742.9762221408345
Mean before =  6352.933333333333
Mean after =  4773.5
Ecart des moyennes = 
4
Date =  2023-04
Std error before =  343.11992905438433
Std error after =  1470.2171041493611
Mean before =  1050.5714285714287
Mean after =  2456.222222222222
Ecart des moyennes = 
5
Date =  2021-12
Std error before =  1998.8416260048725
Std error after =  69.37518768743381
Mean before =  3958.130434782

In [5]:
def find_periods(df, cluster):

    data = df.copy()
    data = data.loc[data["Cluster_8"] == cluster]
    data['DATE_PRISE_EFFET'] = pd.to_datetime(data['DATE_PRISE_EFFET'])
    result = data.groupby(['DATE_PRISE_EFFET', 'TYPE_PROMON'])['ID_ABONNE'].nunique().reset_index()
    result.rename(columns={'ID_ABONNE' : 'NOMBRE_ABONNEMENTS'}, inplace = True)

    
    
    # Calculer la moyenne mobile sur une fenêtre glissante pour chaque TYPE_PROMON
    window_size = 7  # Taille de la fenêtre glissante pour la moyenne mobile
    df['rolling_mean'] = df.groupby('TYPE_PROMON')['NOMBRE_ABONNEMENTS'].transform(lambda x: x.rolling(window_size).mean())

    # Identifier les périodes où une promotion est significativement plus élevée que les autres
    # Par exemple, vous pouvez définir un seuil pour déterminer ce qui constitue une augmentation significative
    seuil = 1.5  # Changer cela selon vos besoins
    promotions_significatives = df[df['NOMBRE_ABONNEMENTS'] > seuil * df['rolling_mean']]

    # Afficher les dates qui délimitent ces périodes
    dates_limites = promotions_significatives['DATE_PRISE_EFFET'].unique()

    print("Les dates limites des périodes où une promotion est significativement plus utilisée que les autres sont :", dates_limites)


    return result


find_periods(dataset, 0)

,DATE_PRISE_EFFET,TYPE_PROMON,NOMBRE_ABONNEMENTS
0,2021-07-30,ODD 15 jours TC,2
1,2021-08-09,ODD 7 jours autre que SG,1
2,2021-08-10,ODD 15 jours EV+,1
3,2021-08-13,ODD 7 jours autre que SG,1
4,2021-08-14,ODD 7 jours autre que SG,1
...,...,...,...
3215,2023-10-29,ODD 21 jours TC,15
3216,2023-10-29,Semaine genéreuse,17
3217,2023-10-30,ODD 21 jours TC,12
3218,2023-10-30,Semaine genéreuse,15


In [35]:
def create_dataset_all_dates(df, cluster):

    data = df.copy()
    data = data.loc[data["Cluster_8"] == cluster]
    """
    data.rename(columns={'DATE_PRISE_EFFET' : 'NOUVEAU_NOM'}, inplace = True)
    data.rename(columns={'DATE_ACTE_REEL' : 'DATE_PRISE_EFFET'}, inplace = True)
    """
    data['DATE_PRISE_EFFET'] = pd.to_datetime(data['DATE_PRISE_EFFET'])
    result = data.groupby(['DATE_PRISE_EFFET', 'TYPE_PROMON'])['ID_ABONNE'].nunique().reset_index()
    result.rename(columns={'ID_ABONNE' : 'NOMBRE_ABONNEMENTS'}, inplace = True)

    
    # Création d'un DataFrame contenant toutes les dates et tous les types promon possibles
    min_date = result['DATE_PRISE_EFFET'].min()
    max_date = result['DATE_PRISE_EFFET'].max()
    all_dates = pd.date_range(start=min_date, end=max_date, freq='D')
    all_promotions = result['TYPE_PROMON'].unique()
    all_possible_combinations = pd.MultiIndex.from_product([all_dates, all_promotions], names=['DATE_PRISE_EFFET', 'TYPE_PROMON'])
    df_all_combinations = pd.DataFrame(index=all_possible_combinations).reset_index()

    # Fusion avec votre DataFrame initial pour remplir les valeurs manquantes
    merged_df = pd.merge(df_all_combinations, result, on=['DATE_PRISE_EFFET', 'TYPE_PROMON'], how='left')

    # Remplacement des valeurs manquantes de NOMBRE_ABONNEMENTS par 0
    merged_df['NOMBRE_ABONNEMENTS'] = merged_df['NOMBRE_ABONNEMENTS'].fillna(0)

    return merged_df
print(create_dataset_all_dates(dataset, 0))
create_dataset_all_dates(dataset, 0)


     DATE_PRISE_EFFET               TYPE_PROMON  NOMBRE_ABONNEMENTS
0          2021-07-30           ODD 15 jours TC                 2.0
1          2021-07-30  ODD 7 jours autre que SG                 0.0
2          2021-07-30          ODD 15 jours EV+                 0.0
3          2021-07-30         Semaine genéreuse                 0.0
4          2021-07-30           ODD 21 jours TC                 0.0
...               ...                       ...                 ...
7411       2023-10-31           ODD 21 jours TC                13.0
7412       2023-10-31           ODD 30 jours TC                 0.0
7413       2023-10-31                   PAS_ODD                 0.0
7414       2023-10-31          ODD 30 jours EV+                 0.0
7415       2023-10-31                    Autres                 0.0

[7416 rows x 3 columns]


,DATE_PRISE_EFFET,TYPE_PROMON,NOMBRE_ABONNEMENTS
0,2021-07-30,ODD 15 jours TC,2.0
1,2021-07-30,ODD 7 jours autre que SG,0.0
2,2021-07-30,ODD 15 jours EV+,0.0
3,2021-07-30,Semaine genéreuse,0.0
4,2021-07-30,ODD 21 jours TC,0.0
...,...,...,...
7411,2023-10-31,ODD 21 jours TC,13.0
7412,2023-10-31,ODD 30 jours TC,0.0
7413,2023-10-31,PAS_ODD,0.0
7414,2023-10-31,ODD 30 jours EV+,0.0


In [33]:
def find_periods(data, cluster):
    df = create_dataset_all_dates(data, cluster)
    # Calculer la moyenne mobile sur une fenêtre glissante pour chaque TYPE_PROMON
    window_size = 7  # Taille de la fenêtre glissante pour la moyenne mobile
    df['MOYENNE_GLISSANTE'] = df.groupby('TYPE_PROMON')['NOMBRE_ABONNEMENTS'].transform(lambda x: x.rolling(window_size).mean())
    print(df.sample(20))
    # Identifier les périodes où une promotion est significativement plus élevée que les autres
    # Par exemple, vous pouvez définir un seuil pour déterminer ce qui constitue une augmentation significative
    seuil = 1.5  # Changer cela selon vos besoins
    promotions_significatives = df[df['NOMBRE_ABONNEMENTS'] > seuil * df['MOYENNE_GLISSANTE']]

    # Afficher les dates qui délimitent ces périodes
    dates_limites = promotions_significatives['DATE_PRISE_EFFET'].unique()

    print("Les dates limites des périodes où une promotion est significativement plus utilisée que les autres sont :", dates_limites)

    return
find_periods(dataset, 2)

     DATE_PRISE_EFFET               TYPE_PROMON  NOMBRE_ABONNEMENTS  \
6152       2023-06-12           ODD 30 jours TC                 0.0   
3377       2022-08-08  ODD 7 jours autre que SG                15.0   
6514       2023-07-22          ODD 30 jours EV+                 0.0   
3439       2022-08-15          ODD 15 jours EV+                 1.0   
3067       2022-07-04          ODD 30 jours EV+                 0.0   
1127       2021-12-01  ODD 7 jours autre que SG                 0.0   
127        2021-08-12          ODD 15 jours EV+                 0.0   
2363       2022-04-17           ODD 30 jours TC                 0.0   
2420       2022-04-23                    Autres                 0.0   
270        2021-08-28           ODD 15 jours TC                 0.0   
6015       2023-05-28         Semaine genéreuse                 1.0   
2064       2022-03-15         Semaine genéreuse                37.0   
6180       2023-06-15                   PAS_ODD                 0.0   
209   

In [36]:
def find_periods(data, cluster):
    df = create_dataset_all_dates(data, cluster)
    # Étape 1 : Segmentez les données en séries temporelles par TYPE_PROMON
    series_by_promotion = {promotion: df[df['TYPE_PROMON'] == promotion] for promotion in df['TYPE_PROMON'].unique()}

    # Étape 2 : Identifier les périodes où le nombre d'abonnements est significativement plus élevé que la moyenne ou la médiane
    periodes_significatives = {}
    for promotion, series in series_by_promotion.items():
        # Calculer la moyenne ou la médiane de la série temporelle
        mean = series['NOMBRE_ABONNEMENTS'].mean()
        # Remplacer mean par median pour utiliser la médiane
        std = series['NOMBRE_ABONNEMENTS'].std()
        seuil = mean + 2 * std  # Choisir un seuil approprié en fonction de votre analyse
        # Identifier les périodes où le nombre d'abonnements dépasse le seuil
        periodes_significatives[promotion] = series[series['NOMBRE_ABONNEMENTS'] > seuil]

    # Étape 3 : Pour chaque période identifiée, déterminez le TYPE_PROMON associé et le nombre total d'abonnements
    for promotion, periodes in periodes_significatives.items():
        for periode in periodes.groupby((periodes['NOMBRE_ABONNEMENTS'] > seuil).cumsum()):
            debut_periode = periode[1].iloc[0]['DATE_PRISE_EFFET']
            fin_periode = periode[1].iloc[-1]['DATE_PRISE_EFFET']
            total_abonnements = periode[1]['NOMBRE_ABONNEMENTS'].sum()
            print(f"Période significative pour {promotion} du {debut_periode} au {fin_periode}, Nombre total d'abonnements : {total_abonnements}")
    return
find_periods(dataset, 2)

Période significative pour ODD 15 jours TC du 2021-11-20 00:00:00 au 2021-11-20 00:00:00, Nombre total d'abonnements : 346.0
Période significative pour ODD 15 jours TC du 2021-12-01 00:00:00 au 2021-12-01 00:00:00, Nombre total d'abonnements : 340.0
Période significative pour ODD 15 jours TC du 2021-12-02 00:00:00 au 2021-12-02 00:00:00, Nombre total d'abonnements : 341.0
Période significative pour ODD 15 jours TC du 2021-12-30 00:00:00 au 2021-12-30 00:00:00, Nombre total d'abonnements : 302.0
Période significative pour ODD 15 jours TC du 2022-04-26 00:00:00 au 2022-04-26 00:00:00, Nombre total d'abonnements : 323.0
Période significative pour ODD 15 jours TC du 2022-05-01 00:00:00 au 2022-05-01 00:00:00, Nombre total d'abonnements : 485.0
Période significative pour ODD 15 jours TC du 2022-05-02 00:00:00 au 2022-05-02 00:00:00, Nombre total d'abonnements : 385.0
Période significative pour ODD 15 jours TC du 2022-05-03 00:00:00 au 2022-05-03 00:00:00, Nombre total d'abonnements : 417.0
